# Separando o dataset com temporal_train_test

In [1]:
# from sktime.forecasting.model_selection import temporal_train_test_split

In [2]:
# seller_id = 100000060
# time_serie = model_df[model_df['Seller ID'] == seller_id]
# time_serie.drop(columns=['Seller ID','Day of Week','Last Week','Last Month'],inplace=True)
# time_serie['Change'] = time_serie['Change'].astype('Int64')
# time_serie

# Lag de um dia na nossa time serie

In [3]:
# time_serie.set_index('Date',inplace=True)

In [4]:
# target = time_serie.pop('Transactioned')
# y = pd.DataFrame(data=target,columns=['Transactioned'])
# y

In [5]:
# time_serie = time_serie.shift(1)
# time_serie.drop(pd.to_datetime('2024-01-30'), inplace=True)
# time_serie

In [6]:
# date_to_pred = pd.to_datetime('2024-11-04')
# time_serie = time_serie[time_serie.index < date_to_pred]
# y = y[y.index<date_to_pred]
# y.drop(pd.to_datetime('2024-01-30'), inplace=True)
# y

In [7]:
# time_serie

In [8]:
# y_train, y_test, X_train, X_test = temporal_train_test_split(y,time_serie,test_size=20)

# Creating forecasting

In [9]:
# from sktime.forecasting.base import ForecastingHorizon
# import numpy as np

# fh = ForecastingHorizon(np.arange(1,len(y_test)+2),is_relative=True)

In [10]:
# from xgboost import XGBRegressor

# # Calculando scale_pos_weight
# n_pos = len(y_train[y_train['Transactioned'] == 1])
# n_neg = len(y_train[y_train['Transactioned'] == 0])
# scale_pos_weight = n_neg / n_pos

# xgb_model = XGBRegressor(
#     n_estimators=100,
#     learning_rate=0.1,
#     max_depth=5,
#     subsample=0.8,
#     colsample_bytree=0.8,
#     random_state=42,
#     scale_pos_weight=scale_pos_weight
# )

In [11]:
# from sktime.forecasting.compose import make_reduction

# forecaster = make_reduction(xgb_model, window_length=12, strategy="recursive")

In [12]:
# X_train.info()

In [13]:
# forecaster.fit(y=y_train,X=X_train)

In [14]:
# y_pred = forecaster.predict(fh=fh,X=X_test)
# y_pred

In [15]:
# from sktime.utils.plotting import plot_series
# plot_series(y_train, y_test, y_pred, labels=["y_train", "y_test", "y_pred"])

In [16]:
# # Análise de erros
# errors = abs(y_test['Transactioned'].values - y_pred.values)
# print("Erro Médio Absoluto (MAE):", errors.mean())

# Iterando em sellers

In [17]:
import pandas as pd
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.forecasting.base import ForecastingHorizon
from xgboost import XGBRegressor
from sktime.forecasting.compose import make_reduction
from sktime.utils.plotting import plot_series
import numpy as np
import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sktime.forecasting.model_selection import SlidingWindowSplitter,ForecastingGridSearchCV, ForecastingOptunaSearchCV
from sktime.split import SingleWindowSplitter
from tqdm import tqdm
from sktime.performance_metrics.forecasting import MeanAbsoluteError
from optuna import Trial
import optuna
from concurrent.futures import ProcessPoolExecutor
import warnings
from sklearn.model_selection import GridSearchCV
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

# Reading DF

In [18]:
path = 'C:/workspace/Desafio Grão Direto IA/Data/model_df.xlsx'
model_df = pd.read_excel(path)

## PCA

In [19]:
# def pca(df):
#     # Define the features variable
#     features = df.columns.tolist()

#     # 1. Normalização das features
#     scaler = StandardScaler()
#     df_scaled = scaler.fit_transform(df)

#     # 2. Aplicar PCA
#     pca = PCA()
#     pca.fit(df_scaled)

#     # 3. Variância explicada
#     explained_variance_ratio = pca.explained_variance_ratio_
#     cumulative_variance = np.cumsum(explained_variance_ratio)

#     # 4. Determinar o número ideal de componentes para atingir, por exemplo, 80% da variância
#     threshold = 0.80
#     num_components = np.argmax(cumulative_variance >= threshold) + 1

#     print(f'Número de componentes para atingir {threshold*100:.0f}% da variância: {num_components}')

#     # 5. Visualizar variância explicada
#     plt.figure(figsize=(12, 7))
#     plt.bar(range(1, len(explained_variance_ratio)+1), explained_variance_ratio, alpha=0.6, color='g', 
#             label='Individual Explained Variance')
#     plt.plot(range(1, len(cumulative_variance)+1), cumulative_variance, marker='o', linestyle='-', color='r', 
#             label='Cumulative Explained Variance')
#     plt.axhline(y=threshold, color='b', linestyle='--', label=f'{threshold*100:.0f}% Variance Threshold')
#     plt.axvline(x=num_components, color='purple', linestyle='--', label=f'{num_components} Components')
#     plt.xlabel('Principal Components')
#     plt.ylabel('Explained Variance')
#     plt.title('Explained Variance by Principal Components')
#     plt.xticks(range(1, len(explained_variance_ratio)+1))
#     plt.legend(loc='upper left')
#     plt.grid(True)
#     plt.show()
#     loadings = pd.DataFrame(pca.components_.T, columns=[f'PC{i+1}' for i in range(len(explained_variance_ratio))],
#                             index=df.columns)
#     loadings = pd.DataFrame(pca.components_.T, columns=[f'PC{i+1}' for i in range(len(explained_variance_ratio))],
#                             index=features)

#     # Exibir os loadings mais importantes para os principais componentes
#     print("Loadings (Pesos das Features nos Componentes Principais):")
#     print(loadings)

#     # Soma absoluta dos loadings para os 4 primeiros PCs
#     top_n_pcs = 4  # Quantos PCs você quer considerar
#     sum_abs_loadings = loadings.iloc[:, :top_n_pcs].abs().sum(axis=1)

#     # Adiciona a soma como uma nova coluna no DataFrame original das features
#     loadings['Sum_Absolute_Loadings_Top4'] = sum_abs_loadings

#     # Ordena as features por importância (soma decrescente)
#     sorted_features = loadings['Sum_Absolute_Loadings_Top4'].sort_values(ascending=False)

#     # Exibir as features mais importantes
#     print("Soma Absoluta dos Pesos nos 4 Primeiros PCs (por Feature):")
#     print(sorted_features)

#     # Identificar as features mais importantes para os primeiros componentes principais
#     top_features = {}
#     for i in range(num_components):
#         pc_loadings = loadings[f'PC{i+1}'].abs().sort_values(ascending=False)
#         top_features[f'PC{i+1}'] = pc_loadings.head(3).index.tolist()

#     print("\nFeatures mais importantes para cada componente principal:")
#     for pc, features in top_features.items():
#         print(f'{pc}: {features}')

# pca_df = model_df[model_df['Seller ID'] == 100000060]
# pca_df.set_index('Date',inplace=True)
# pca_df = pca_df.drop(columns=['Transactioned','Seller ID',])

# pca(pca_df)

## Modelo

## GridSearch

In [20]:
def process_seller_gs(seller_id):
    #print(f"Processando Seller ID: {seller_id}")
    
    # Filtrar os dados para o Seller ID atual
    time_serie = model_df[model_df['Seller ID'] == seller_id]
    time_serie = time_serie.drop(columns=['Seller ID','Day of Week'])
    
    # Preparação da série temporal
    time_serie.set_index('Date', inplace=True)
    target = time_serie.pop('Transactioned')
    y = pd.DataFrame(data=target, columns=['Transactioned'])
    time_serie = time_serie.shift(1)
    
    # Remover valores fora do intervalo de datas desejado
    date_to_pred = pd.to_datetime('2024-11-04')
    first_date = pd.to_datetime('2024-01-30')
    mask = (time_serie.index > first_date) & (time_serie.index < date_to_pred)
    time_serie = time_serie.loc[mask]
    y = y.loc[mask]

    # Dividir os dados em treino e teste
    y_train, y_test, X_train, X_test = temporal_train_test_split(y, time_serie, test_size=2)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    X_train = pd.DataFrame(data=X_train_scaled, columns=X_train.columns, index=X_train.index)
    X_test = pd.DataFrame(data=X_test_scaled, columns=X_test.columns, index=X_test.index)
    
    # Configuração do Forecasting Horizon
    fh = ForecastingHorizon([3], is_relative=True)
    
    # Calculando scale_pos_weight
    n_pos = len(y_train[y_train['Transactioned'] == 1])
    n_neg = len(y_train[y_train['Transactioned'] == 0])
    scale_pos_weight = n_neg / n_pos if n_pos > 0 else 1
    
    # Configuração do modelo e busca
    param_grid = {
        'n_estimators': [100, 500],
        'learning_rate': [0.01, 0.1],
        'max_depth': [3, 6],
        'subsample': [0.1, 0.3],
        'colsample_bytree': [0.6, 0.8]
    }

    xgb_model = XGBRegressor(
        objective='reg:squarederror',
        random_state=42,
        scale_pos_weight=scale_pos_weight
    )
    
    # Criar o modelo
    forecaster = make_reduction(xgb_model, window_length=12, strategy="recursive")

    cv = SingleWindowSplitter(window_length=len(y)-2,fh=2)

    # Criar o ForecastingGridSearchCV
    grid_search = ForecastingGridSearchCV(
        forecaster=forecaster,
        param_grid=param_grid,
        cv=cv,
        scoring=MeanAbsoluteError()
    )

    #Treinando modelo
    grid_search.fit(y=y_train, X=X_train)
    
    # Previsão
    y_pred = grid_search.predict(fh=fh, X=X_test)
    # y_pred_04 = y_pred.loc['2024-11-04']

    # # Análise de erros
    # errors = abs(y_test['Transactioned'].values - y_pred.values)
    # mae = errors.mean()
    # print(f"Erro Médio Absoluto (MAE) para Seller ID {seller_id}: {mae}")
    
    # # Plot das séries
    # plot_series(y_train, y_test, y_pred, labels=["y_train", "y_test", "y_pred"])
    # plt.title(f"Previsão para Seller ID: {seller_id}")
    # plt.show()
    
    # Armazenar os resultados
    return{
        "Seller ID": seller_id,
        "Date": y_pred.index.values[0],  # Extract the single date value
        "Score": y_pred['Transactioned'].values[0],  # Extract the single score value
        # "MAE": mae
        "Params": grid_search.cv_results_
    }

# Lista de Seller IDs
seller_ids = model_df['Seller ID'].unique()

# Resultados para armazenar os erros por vendedor
results = []

# Criar a barra de progresso manualmente
with ThreadPoolExecutor() as executor:
    # Usar tqdm para iterar nos vendedores enquanto processa
    for result in tqdm(executor.map(process_seller_gs, seller_ids), total=len(seller_ids), desc="Processando Sellers"):
        results.append(result)

# Criar DataFrame com os resultados
results_df = pd.DataFrame(results)
# results_df

72 minutos e 33,5 segundos

## OPTUNA

In [ ]:
def process_seller_optuna(seller_id):
    warnings.simplefilter(action="ignore", category=FutureWarning)
    #print(f"Processando Seller ID: {seller_id}")
    
    # Filtrar os dados para o Seller ID atual
    time_serie = model_df[model_df['Seller ID'] == seller_id]
    time_serie = time_serie.drop(columns=['Seller ID','Day of Week'])
    
    # Preparação da série temporal
    time_serie.set_index('Date', inplace=True)
    target = time_serie.pop('Transactioned')
    y = pd.DataFrame(data=target, columns=['Transactioned'])
    time_serie = time_serie.shift(1)
    
    # Remover valores fora do intervalo de datas desejado
    date_to_pred = pd.to_datetime('2024-11-04')
    first_date = pd.to_datetime('2024-01-30')
    mask = (time_serie.index > first_date) & (time_serie.index < date_to_pred)
    time_serie = time_serie.loc[mask]
    y = y.loc[mask]

    # Dividir os dados em treino e teste
    y_train, y_test, X_train, X_test = temporal_train_test_split(y, time_serie, test_size=2)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    X_train = pd.DataFrame(data=X_train_scaled, columns=X_train.columns, index=X_train.index)
    X_test = pd.DataFrame(data=X_test_scaled, columns=X_test.columns, index=X_test.index)
    
    # Configuração do Forecasting Horizon
    fh = ForecastingHorizon([3], is_relative=True)
    
    # Calculando scale_pos_weight
    n_pos = len(y_train[y_train['Transactioned'] == 1])
    n_neg = len(y_train[y_train['Transactioned'] == 0])
    scale_pos_weight = n_neg / n_pos if n_pos > 0 else 1
    
    # Configuração do modelo e busca
    param_grid = {
        'n_estimators': [100, 500],
        'learning_rate': [0.01, 0.1],
        'max_depth': [3, 6],
        'subsample': [0.1, 0.3],
        'colsample_bytree': [0.6, 0.8]
    }

    xgb_model = XGBRegressor(
        objective='reg:squarederror',
        random_state=42,
        scale_pos_weight=scale_pos_weight
    )
    
    # Criar o modelo
    forecaster = make_reduction(xgb_model, window_length=12, strategy="recursive")

    cv = SingleWindowSplitter(window_length=len(y)-2,fh=2)

    # Criar o ForecastingGridSearchCV
    grid_search = ForecastingOptunaSearchCV(
        forecaster=forecaster,
        param_grid=param_grid,
        cv=cv,
        n_evals=10,
    )

    #Treinando modelo
    grid_search.fit(y=y_train, X=X_train)
    
    # Previsão
    y_pred = grid_search.predict(fh=fh, X=X_test)
    # y_pred_04 = y_pred.loc['2024-11-04']

    # # Análise de erros
    # errors = abs(y_test['Transactioned'].values - y_pred.values)
    # mae = errors.mean()
    # print(f"Erro Médio Absoluto (MAE) para Seller ID {seller_id}: {mae}")
    
    # # Plot das séries
    # plot_series(y_train, y_test, y_pred, labels=["y_train", "y_test", "y_pred"])
    # plt.title(f"Previsão para Seller ID: {seller_id}")
    # plt.show()
    
    # Armazenar os resultados
    return{
        "Seller ID": seller_id,
        "Date": y_pred.index.values[0],  # Extract the single date value
        "Score": y_pred['Transactioned'].values[0],  # Extract the single score value
        # "MAE": mae
        "Params": grid_search.cv_results_
    }

# Lista de Seller IDs
seller_ids = model_df['Seller ID'].unique()

# Resultados para armazenar os erros por vendedor
results = []

# Criar a barra de progresso manualmente
with ThreadPoolExecutor() as executor:
    # Usar tqdm para iterar nos vendedores enquanto processa
    for result in tqdm(executor.map(process_seller_optuna, seller_ids), total=len(seller_ids), desc="Processando Sellers"):
        results.append(result)

# Criar DataFrame com os resultados
results_df = pd.DataFrame(results)
# results_df

In [ ]:
media = results_df['Score'].mean()
media

In [47]:
users_that_made_transaction_on_target_date_04 = [
    100000060,
    100000141,
    100000192,
    100000310,
    100000348,
    100001098,
    100001362,
    100001525,
    100002035,
    100002271,
    100002332,
    100002339,
    100002402,
    100002426,
]

In [48]:
users_that_made_transaction_on_target_date_05 = [
    100000180,
    100000037,
    100001254,
    10000088,
    100001203,
    100001098,
    100001599,
    100001885,
    100000608,
    100000151,
    100000638,
]

In [ ]:
first_200_predictions = results_df.sort_values(
    ascending=False, by="Score"
).head(200)


first_200_predictions = first_200_predictions[
    first_200_predictions["Seller ID"].isin(users_that_made_transaction_on_target_date_04)
]

In [57]:
first_200_predictions.to_excel('C:/workspace/Desafio Grão Direto IA/Data/params_gscv.xlsx',index=False)

In [ ]:
results_df

# Coisas a fazer
- PCA |
- GRIDSEARCH
- OPTUNASEARCH
- CONCURRENT FUTURES V
- Hierarchical Clustering |

## Hierarchical Clustering

Tomar cuidado com features categoricas, até mesmo transformadas em int

In [51]:
# from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
# import matplotlib.pyplot as plt

# model_cl = model_df.drop(columns=['Seller ID', 'Transactioned'])
# model_cl = model_cl.set_index('Date')
# scaler = StandardScaler()
# model_clp = scaler.fit_transform(model_cl)
# model_clp = pd.DataFrame(model_clp, columns=model_cl.columns)
# model_clp = model_clp.transpose()

# linked = linkage(model_clp, method='ward', metric='euclidean')

# df_linked = pd.DataFrame(linked, columns=['c1', 'c2', 'distance', 'size'])
# df_linked[['c1', 'c2', 'size']] = df_linked[['c1', 'c2', 'size']].astype('Int64')

# plt.figure(figsize=(15,5))

# dendrogram(linked, orientation='top', labels=model_clp.index, distance_sort='descending', show_leaf_counts=True)
# plt.xlabel('Features')
# plt.ylabel('Wards')
# plt.show()